In [ ]:
import time
import awkward as ak
import numpy as np
from coffea.nanoevents import NanoEventsFactory, BaseSchema, NanoAODSchema

fname_wobib = "/nfs_scratch/slomte/MuColl/runjobs10bib/singleHiggs/bib100/analysis/JetHistogramsHbb_sig_only_bib100_2mev_250ps_ak5_10kEvents.root"
fname_wbib = "/nfs_scratch/slomte/MuColl/runjobs10bib/singleHiggs/bib100/analysis/JetHistogramsHbb_sig_and_BIB_bib100_2mev_250ps_ak5_10kEvents.root"

eve_gen = NanoEventsFactory.from_root(
    fname_wobib,
    #metadata={"dataset":"gen level"},
    "JetHistogramGenJetTuple;1",
    schemaclass=BaseSchema,
).events()
eve_reco_wobib = NanoEventsFactory.from_root(
    fname_wobib,
    #metadata={"dataset":"w/o BIB"},
    "JetHistogramRecoJetTuple;1",
    schemaclass=BaseSchema,
).events()
eve_reco_wbib = NanoEventsFactory.from_root(
    fname_wbib,
    #metadata={"dataset":"w BIB"},
    "JetHistogramRecoJetTuple;1",
    schemaclass=BaseSchema,
).events()


In [ ]:
from coffea import processor
from coffea.nanoevents.methods import candidate, vector, nanoaod
from coffea import hist
from functools import partial
import math
import numba
import pickle
import re

import matplotlib.pyplot as plt
import scipy.optimize as opt;


def selectJets(events):
    Jets = ak.zip(
    {
        "x": events.jmox,
        "y": events.jmoy,
        "z": events.jmoz,
        "t": events.jene,
        "cost": events.jcost,
    },
        with_name="LorentzVector",
        behavior=vector.behavior,
    )        
    sort_inds = ak.argsort(Jets.pt, ascending=False)
    Jets = Jets[sort_inds]
    return Jets

def findGenRecoMatch(i_gjets, i_rjets):
    f_dR_gj_rj = i_gjets.metric_table(i_rjets, axis=-1)
    o_gjets = i_gjets[ak.any(f_dR_gj_rj<0.5, axis=-1)]
    f_dR_new = o_gjets.metric_table(i_rjets, axis=-1)
    f_argmin = ak.argmin(f_dR_new, axis=-1)
    o_rjets = i_rjets[f_argmin]
    o_gjrj_pairs = ak.zip({'gJet': o_gjets, 'rJet': o_rjets})
    return o_gjrj_pairs



class MyProcessor(processor.ProcessorABC):
    def __init__(self):
        # Categories
        dataset_axis = hist.Cat("dataset", "")#gen, reco with, or reco without BIB
        evtlabel_axis = hist.Cat("evtlabel", "")#describing event, e.g 2jets in event
        jetIndex_axis = hist.Cat("jetIndex", "")#leading or subleading jet in PT
        cut_axis = hist.Cat("cut", "")#Acceptance cut
        
        # Variables
        n_axis = hist.Bin("n", "N", 20, 0, 20)
        pt_axis = hist.Bin("pt", r"$p_{T}$ [GeV]", 30, 1.0, 300.)
        eta_axis = hist.Bin("eta", r"$\eta$", 50, -5.0, 5.0)
        phi_axis = hist.Bin("phi", r"$\phi$", 50, -5.0, 5.0)
        energy_axis = hist.Bin("energy", r"$E$ [GeV]", 50, 1.0, 500.)        
        mass_axis = hist.Bin("mass", r"mass [GeV]", 40, 1.0, 200.)
        cost_axis = hist.Bin("cost", r"$cos\theta$", 22, -1.1, 1.1)
        dr_axis = hist.Bin("dr", r"$\Delta R$", 50, 0., 5.0)
        
        
        ### Accumulator for holding histograms
        self._accumulator = processor.dict_accumulator(
            {
                "EventCount": processor.value_accumulator(int),
                "cutflow" : processor.defaultdict_accumulator(partial(processor.defaultdict_accumulator, int)),
                
                
                # jet kinematics
                "Jets_N" : hist.Hist("Events", dataset_axis, cut_axis, n_axis),
                "Jet_pt": hist.Hist("Events", dataset_axis, cut_axis, jetIndex_axis, pt_axis),
                "Jet_eta": hist.Hist("Events", dataset_axis, cut_axis, jetIndex_axis, eta_axis),
                "Jet_phi": hist.Hist("Events", dataset_axis, cut_axis, jetIndex_axis, phi_axis),
                "Jet_energy": hist.Hist("Events", dataset_axis, cut_axis, jetIndex_axis, energy_axis),
                "Jet_mass": hist.Hist("Events", dataset_axis, cut_axis, jetIndex_axis, mass_axis),
                "Jet_cosTheta": hist.Hist("Events", dataset_axis, cut_axis, jetIndex_axis, cost_axis),
                
                # di-jet kinematics
                "dijet_mass": hist.Hist("Events", dataset_axis, evtlabel_axis, mass_axis),
                "trijet_mass": hist.Hist("Events", dataset_axis, evtlabel_axis, mass_axis),
                "Inv_Mass": hist.Hist("Events", dataset_axis, evtlabel_axis, mass_axis),                
                "H_pt": hist.Hist("Events", dataset_axis, cut_axis, pt_axis),
                "H_eta": hist.Hist("Events", dataset_axis, cut_axis, eta_axis),
                "H_phi": hist.Hist("Events", dataset_axis, cut_axis, phi_axis),
                "H_energy": hist.Hist("Events", dataset_axis, cut_axis, energy_axis),
                
                "dR_j1j2": hist.Hist("Events", dataset_axis, cut_axis, dr_axis),
                
            }
        )

    @property
    def accumulator(self):
        return self._accumulator

    def process(self, events, datalabel):
        #dataset = events.metadata['dataset']
        output = self.accumulator.identity()

        #------Code starts here------#        
        #--- jet ---#
        jets = ak.zip(
            {
                "x": events.jmox,
                "y": events.jmoy,
                "z": events.jmoz,
                "t": events.jene,
                "cost": events.jcost,
            },
            with_name="LorentzVector",
            behavior=vector.behavior,
        )        
        sort_inds = ak.argsort(jets.pt, ascending=False)
        jets = jets[sort_inds]
        
        output["Jets_N"].fill(dataset=datalabel, n=ak.num(jets, axis=1), cut='nocut')
        output["Jets_N"].fill(dataset=datalabel, n=ak.num(jets[jets.pt>20], axis=1), cut='pt20')
        output["Jets_N"].fill(dataset=datalabel, n=ak.num(jets[(jets.pt>20)&(abs(jets.cost)<0.98)], axis=1), cut='pt20cost0p98')
        output["Jets_N"].fill(dataset=datalabel, n=ak.num(jets[(jets.pt>20)&(abs(jets.cost)<0.9)], axis=1), cut='pt20cost0p9')
        output["Jets_N"].fill(dataset=datalabel, n=ak.num(jets[(jets.pt>20)&(abs(jets.cost)<0.8)], axis=1), cut='pt20cost0p8')
        
        
        #Object selections (acceptance cuts)
        PtTheta_cut = (jets.pt>20) & (abs(jets.cost)<0.8)
        goodjets = jets[PtTheta_cut]
        #print(goodjets.pt)
        
        #good jet kinematics
        output["Jets_N"].fill(dataset=datalabel, n=ak.num(goodjets, axis=1), cut='pt20cost0p8')
        output["Jet_pt"].fill(dataset=datalabel, pt=ak.flatten(goodjets.pt), jetIndex='All', cut='pt20cost0p8')
        output["Jet_eta"].fill(dataset=datalabel, eta=ak.flatten(goodjets.eta), jetIndex='All', cut='pt20cost0p8')
        output["Jet_phi"].fill(dataset=datalabel, phi=ak.flatten(goodjets.phi), jetIndex='All', cut='pt20cost0p8')
        output["Jet_mass"].fill(dataset=datalabel, mass=ak.flatten(goodjets.mass), jetIndex='All', cut='pt20cost0p8')
        output["Jet_energy"].fill(dataset=datalabel, energy=ak.flatten(goodjets.energy), jetIndex='All', cut='pt20cost0p8')
        output["Jet_cosTheta"].fill(dataset=datalabel, cost=ak.flatten(goodjets.cost), jetIndex='All', cut='pt20cost0p8')
        
        # ----------------------------------------------------#
        # -------------- Jet momentum Correction -------------#
        # ----------------------------------------------------#
        #Why: To recover energy lost in reconstruction. 
        #How: (1) for every gen jet, find a reco matched jet with min_dR<0.5
        #     (2) In pT - Theta regions, find average and std deviation of gen jets 
        
        # Step (1): 
        #The function 'findGenRecoMatch' finds gen and reco jet matches.
        #The inputs are genjets and recojets arrays, output is zipped array with 1st index 'gJet' and 2nd index 'rJet' which are LorentzVectors
        #The genjets which don't have a reco jet within dR<0.5, those jets are removed from the list. So the zipped pairs have same strucure. 
        
        recojets = selectJets(eve_reco_wobib)
        genjets = selectJets(eve_gen)
        gen_reco_pairs = findGenRecoMatch(genjets, recojets)
        
        gpt = ak.fill_none(gen_reco_pairs.gJet.pt, -10000000)
        geta = ak.fill_none(abs(gen_reco_pairs.gJet.eta), -10000000)
        reta = ak.fill_none(abs(gen_reco_pairs.rJet.eta), -10000000)
        gjets = gen_reco_pairs.gJet
        rjets = gen_reco_pairs.rJet
        
        # Step (2):
        #for each Eta region, find mean and s.d. of pT of genjets
        # pT bins: [20,40,...200] --> 9 bins
        # |n| bins: [0,0.5,1.0,1.5,2.0,2.5] --> 5 bins
        
        etaregions = np.linspace(0., 2.5, num=6)
        ptregions = np.linspace(10, 200, num=20)
        gjets_meanPT_reg = np.ones((5,19))
        gjets_stdPT_reg = np.ones((5,19))
        rjets_meanPT_reg = np.ones((5,19))
        
        for j in range(0,5):
            etareg = (geta>=etaregions[j]) & (geta<etaregions[j+1])
            for i in range(0,19):
                ptreg = (gpt>=ptregions[i]) & (gpt<ptregions[i+1])
                
                # fills mean and s.d of pT of genjets for each eta and pT region. Size: 5*19 
                gjets_meanPT_reg[j][i] = ak.mean(gjets.pt[ptreg & etareg], axis=None)
                gjets_stdPT_reg[j][i] = ak.std(gjets.pt[ptreg & etareg], axis=None)        
                rjets_meanPT_reg[j][i] = ak.mean(rjets.pt[ptreg & etareg], axis=None)
        
        #print(gjets_meanPT_reg[2,:])
        #print(gjets_stdPT_reg[2,:])
        
        # Step (3):
        # Try plotting genjets average pT vs recojets average pT for various genjet-eta regions
        # for reco pT, we find average pT in each genjet's pT-eta region 
        # the correction factor is the ratio of the two, which should then be applied to each component of the recojet 4-momentum.
                
        #plt.plot(rjets_meanPT_reg[0,:],gjets_meanPT_reg[0,:], ".", label="reg1")
        #plt.plot(rjets_meanPT_reg[1,:],gjets_meanPT_reg[1,:], ".", label="reg2")
        #plt.plot(rjets_meanPT_reg[2,:],gjets_meanPT_reg[2,:], ".", label="reg3")
        #plt.plot(rjets_meanPT_reg[3,:],gjets_meanPT_reg[3,:], ".", label="reg4")
        #plt.plot(rjets_meanPT_reg[4,:],gjets_meanPT_reg[4,:], ".", label="reg5")
        
        # We fit a linear function
        def fitted_function(x, a, b, c):
             return a*(x**b) + c
        
        optimizedParameters = np.ones((5,3))
        #covarianceParameters = ? check its array structure, also check if needed for future use, like for calculating std deviation or something..
        op, cov = 0, 0
        for e in range(0,5):
            recoPT_avg = rjets_meanPT_reg[e,:]
            genPT_avg = gjets_meanPT_reg[e,:]
            op, cov = opt.curve_fit(fitted_function, recoPT_avg, genPT_avg)        
            optimizedParameters[e] = op
            #covarianceParameters[e] = cov        
        
        #----> We access the fitted function using: fitted_function(x, *optimizedParameters[eta_region])
        # We can plot the fitted curve and original pT of gen & reco jets
        '''
        fig, ax = plt.subplots(1)
        plt.plot(rjets_meanPT_reg[0,:], gjets_meanPT_reg[0,:], ".", label="Data")
        plt.plot(rjets_meanPT_reg[0,:], fitted_function(rjets_meanPT_reg[0,:], *optimizedParameters[0]), label="fit")
        plt.legend()
        ax.set_xlabel(r'reco jet $p_{T}$ [GeV]', fontsize=13)
        ax.set_ylabel(r'gen jet $p_{T}$ [GeV]', fontsize=13)
        process = plt.text(1.0, 1.0, r"$H \rightarrow b \bar{b}$ (1.5TeV)", fontsize=12, horizontalalignment="right", verticalalignment="bottom", transform=ax.transAxes)
        MC = plt.text(0.0, 1.0, r"$\bf{Muon Collider}$ Preliminary", fontsize=12, horizontalalignment="left", verticalalignment="bottom", transform=ax.transAxes)
        Sidenote1 = plt.text(0.75, 0.60,r"$|\eta|<0.5$", fontsize=12, transform=ax.transAxes)
        plt.savefig('transfer_func_etareg1.png')
        
        fig, ax = plt.subplots(1)
        plt.plot(rjets_meanPT_reg[3,:], gjets_meanPT_reg[3,:], ".", label="Data")
        plt.plot(rjets_meanPT_reg[3,:], fitted_function(rjets_meanPT_reg[3,:], *optimizedParameters[3]), label="fit")
        plt.legend()
        ax.set_xlabel(r'reco jet $p_{T}$ [GeV]', fontsize=13)
        ax.set_ylabel(r'gen jet $p_{T}$ [GeV]', fontsize=13)
        process = plt.text(1.0, 1.0, r"$H \rightarrow b \bar{b}$ (1.5TeV)", fontsize=12, horizontalalignment="right", verticalalignment="bottom", transform=ax.transAxes)
        MC = plt.text(0.0, 1.0, r"$\bf{Muon Collider}$ Preliminary", fontsize=12, horizontalalignment="left", verticalalignment="bottom", transform=ax.transAxes)
        Sidenote1 = plt.text(0.75, 0.60,r"$1.5<|\eta|<2$", fontsize=12, transform=ax.transAxes)
        plt.savefig('transfer_func_etareg4.png')
        '''
        #print(gjets.pt[(geta>=0) & (geta<0.5)])
        #print(rjets.pt[(reta>=0) & (reta<0.5)])
        #print(fitted_function(rjets.pt[(reta>=0) & (reta<0.5)], *optimizedParameters[0]))
        
        
        # Step (4):
        # Find scale factor for each reco jet using the transfer function        
        # scale factor is corr_reco_jet PT / reco_jet PT for various eta regions:
        scale_factors = ak.where((reta>=0) & (reta<0.5), (fitted_function(ak.mask(rjets.pt, (reta>=0) & (reta<0.5)), *optimizedParameters[0]))/(ak.mask(rjets.pt, (reta>=0) & (reta<0.5))), 0)
        scale_factors = ak.where((reta>=0.5) & (reta<1.0), (fitted_function(ak.mask(rjets.pt, (reta>=0.5) & (reta<1.0)), *optimizedParameters[1]))/(ak.mask(rjets.pt, (reta>=0.5) & (reta<1.0))), scale_factors)
        scale_factors = ak.where((reta>=1.0) & (reta<1.5), (fitted_function(ak.mask(rjets.pt, (reta>=1.0) & (reta<1.5)), *optimizedParameters[2]))/(ak.mask(rjets.pt, (reta>=1.0) & (reta<1.5))), scale_factors)
        scale_factors = ak.where((reta>=1.5) & (reta<2.0), (fitted_function(ak.mask(rjets.pt, (reta>=1.5) & (reta<2.0)), *optimizedParameters[3]))/(ak.mask(rjets.pt, (reta>=1.5) & (reta<2.0))), scale_factors)
        scale_factors = ak.where((reta>=2.0) & (reta<2.5), (fitted_function(ak.mask(rjets.pt, (reta>=2.0) & (reta<2.5)), *optimizedParameters[4]))/(ak.mask(rjets.pt, (reta>=2.0) & (reta<2.5))), scale_factors)
        
        corr_reco_Jets = ak.zip(
            {
                "x": scale_factors*rjets.x,
                "y": scale_factors*rjets.y,
                "z": scale_factors*rjets.z,
                "t": scale_factors*rjets.t,
                "cost": rjets.cost,
            },
            with_name="LorentzVector",
            behavior=vector.behavior,
        )        

        # create a 3rd record with corr_reco_jets in gen_reco_pairs.
        #gen_reco_creco_jets = ak.zip({'gJet': gjets, 'rJet': rjets, 'crJet': corr_reco_Jets})
        # --------------------------------------------------#
        # --------------------------------------------------#
                
        #plot kinematics of corrected reco jets
        PtTheta_cut_rj = (corr_reco_Jets.pt>20) & (abs(corr_reco_Jets.cost)<0.8)
        good_crjets = corr_reco_Jets[PtTheta_cut_rj]
        output["Jets_N"].fill(dataset=datalabel, n=ak.num(good_crjets, axis=1), cut='crjets_pt20cost0p8')
        output["Jet_pt"].fill(dataset=datalabel, pt=ak.flatten(good_crjets.pt), jetIndex='All', cut='crjets_pt20cost0p8')
        output["Jet_eta"].fill(dataset=datalabel, eta=ak.flatten(good_crjets.eta), jetIndex='All', cut='crjets_pt20cost0p8')
        output["Jet_phi"].fill(dataset=datalabel, phi=ak.flatten(good_crjets.phi), jetIndex='All', cut='crjets_pt20cost0p8')
        output["Jet_mass"].fill(dataset=datalabel, mass=ak.flatten(good_crjets.mass), jetIndex='All', cut='crjets_pt20cost0p8')
        output["Jet_energy"].fill(dataset=datalabel, energy=ak.flatten(good_crjets.energy), jetIndex='All', cut='crjets_pt20cost0p8')
        output["Jet_cosTheta"].fill(dataset=datalabel, cost=ak.flatten(good_crjets.cost), jetIndex='All', cut='crjets_pt20cost0p8')
        
        # -------------- Acceptance efficiency -------------#
        ptcut = 20
        for costhetacut in [0.8,0.85,0.9]:
            jets_wPt_cut = jets[(jets.pt>ptcut)]
            jets_wCost_cut = jets[(abs(jets.cost)<costhetacut)]
            jets_wPtCost_cut = jets[(jets.pt>ptcut) & (abs(jets.cost)<costhetacut)]
            numEvts_wPt_cut = ak.sum(ak.num(jets_wPt_cut.pt, axis=1)>1) #atleast 2jets per evt which satisfies pt cut
            numEvts_wPtCost_cut = ak.sum(ak.num(jets_wPtCost_cut.pt, axis=1)>1) #atleast 2jets per evt which satisfies pt and cosTheta cuts
            print(f'Acceptance Efficiency (pT>{ptcut}, |costheta|<{costhetacut}) = ' ,numEvts_wPtCost_cut,'/',numEvts_wPt_cut,'=',numEvts_wPtCost_cut/numEvts_wPt_cut)
            
        output["Jet_cosTheta"].fill(dataset=datalabel, cost=(jets_wPt_cut.cost[ak.num(jets_wPt_cut.pt, axis=1)>1][:,0]), jetIndex='j1', cut='pt20')
        output["Jet_cosTheta"].fill(dataset=datalabel, cost=(jets_wPt_cut.cost[ak.num(jets_wPt_cut.pt, axis=1)>1][:,1]), jetIndex='j2', cut='pt20')
        output["Jet_cosTheta"].fill(dataset=datalabel, cost=(jets_wPt_cut.cost[ak.num(jets_wPt_cut.pt, axis=1)>2][:,2]), jetIndex='j3', cut='pt20')
        output["Jet_pt"].fill(dataset=datalabel, pt=(jets_wCost_cut.pt[ak.num(jets_wCost_cut.pt, axis=1)>1][:,0]), jetIndex='j1', cut='cost0p8')
        output["Jet_pt"].fill(dataset=datalabel, pt=(jets_wCost_cut.pt[ak.num(jets_wCost_cut.pt, axis=1)>1][:,1]), jetIndex='j2', cut='cost0p8')
        output["Jet_pt"].fill(dataset=datalabel, pt=(jets_wCost_cut.pt[ak.num(jets_wCost_cut.pt, axis=1)>2][:,2]), jetIndex='j3', cut='cost0p8')
        
        #corrected reco jets efficiency:
        
        ptcut = 20
        for costhetacut in [0.8,0.85,0.9]:
            jets_wPt_cut = corr_reco_Jets[(corr_reco_Jets.pt>ptcut)]
            jets_wCost_cut = corr_reco_Jets[(abs(corr_reco_Jets.cost)<costhetacut)]
            jets_wPtCost_cut = corr_reco_Jets[(corr_reco_Jets.pt>ptcut) & (abs(corr_reco_Jets.cost)<costhetacut)]
            numEvts_wPt_cut = ak.sum(ak.num(jets_wPt_cut.pt, axis=1)>1) #atleast 2jets per evt which satisfies pt cut
            numEvts_wPtCost_cut = ak.sum(ak.num(jets_wPtCost_cut.pt, axis=1)>1) #atleast 2jets per evt which satisfies pt and cosTheta cuts
            print(f'Reco Acceptance Efficiency (pT>{ptcut}, |costheta|<{costhetacut}) = ' ,numEvts_wPtCost_cut,'/',numEvts_wPt_cut,'=',numEvts_wPtCost_cut/numEvts_wPt_cut)
                
        # --------------------------------------------------#
        
        # -------------- Reconstruction efficiency -------------#
        # find number of events with atleast one jet with in a pT or theta region at gen and reco level
        '''
        recojets = selectJets(eve_reco_wobib)
        genjets = selectJets(eve_gen)
        recojets = recojets[recojets.pt>20]
        genjets = genjets[genjets.pt>20]
        
        PT_bins = [0.,20.,40.,60.,80.,100.,120.,140.,160.,180.,200.]
        cosTheta_bins = [0.0,0.2,0.4,0.6,0.8,1.0]
        
        #in cosTheta
        for i in range(0,5):
            if ((genjets.pt>=PT_bins[i]) & (genjets.pt<PT_bins[i+1]) ):
        '''
        # --------------------------------------------------#
        
        
        #------ Higgs reconstruction from dijet or trijet ------#   
        ptcut=20
        costhetacut=0.8
        goodjets = jets[(jets.pt>ptcut) & (abs(jets.cost)<costhetacut)]
        jet1 = ak.mask(goodjets, ak.num(goodjets, axis=1)==2)[:,0]
        jet2 = ak.mask(goodjets, ak.num(goodjets, axis=1)==2)[:,1]
        dijet = jet1.add(jet2)
        jet1 = ak.mask(goodjets, ak.num(goodjets, axis=1)>2)[:,0]
        jet2 = ak.mask(goodjets, ak.num(goodjets, axis=1)>2)[:,1]
        jet3 = ak.mask(goodjets, ak.num(goodjets, axis=1)>2)[:,2]
        trijet = (jet1.add(jet2)).add(jet3)        
        dijet_mass = ak.fill_none(dijet.mass, 0)
        trijet_mass = ak.fill_none(trijet.mass, 0)
        dijet_pt = ak.fill_none(dijet.pt,0)
        trijet_pt = ak.fill_none(trijet.pt,0)
        dijet_energy = ak.fill_none(dijet.energy,0)
        trijet_energy = ak.fill_none(trijet.energy,0)
        dijet_eta = ak.fill_none(dijet.eta,1000)
        trijet_eta = ak.fill_none(trijet.eta,1000)
        dijet_phi = ak.fill_none(dijet.phi,1000)
        trijet_phi = ak.fill_none(trijet.phi,1000)
        
        # reco Higgs kinematics
        output['dijet_mass'].fill(dataset=datalabel, evtlabel='=2j evts', mass=dijet_mass)
        output['trijet_mass'].fill(dataset=datalabel, evtlabel='>2j evts', mass=trijet_mass)        
        output['Inv_Mass'].fill(dataset=datalabel, evtlabel='>=2j evts', mass=dijet_mass)
        output['Inv_Mass'].fill(dataset=datalabel, evtlabel='>=2j evts', mass=trijet_mass)
        # for >=2jet events
        output['H_pt'].fill(dataset=datalabel, pt=dijet_pt, cut='pt20cost0p8')
        output['H_pt'].fill(dataset=datalabel, pt=trijet_pt, cut='pt20cost0p8')
        output['H_eta'].fill(dataset=datalabel, eta=dijet_eta, cut='pt20cost0p8')
        output['H_eta'].fill(dataset=datalabel, eta=trijet_eta, cut='pt20cost0p8')
        output['H_phi'].fill(dataset=datalabel, phi=dijet_phi, cut='pt20cost0p8')
        output['H_phi'].fill(dataset=datalabel, phi=trijet_phi, cut='pt20cost0p8')
        output['H_energy'].fill(dataset=datalabel, energy=dijet_energy, cut='pt20cost0p8')
        output['H_energy'].fill(dataset=datalabel, energy=trijet_energy, cut='pt20cost0p8')
        
        # Higgs jet constituent kinematics (j1,j2,j3) 
        jet1 = ak.mask(goodjets, ak.num(goodjets, axis=1)>1)[:,0]
        jet2 = ak.mask(goodjets, ak.num(goodjets, axis=1)>1)[:,1]
        jet3 = ak.mask(goodjets, ak.num(goodjets, axis=1)>2)[:,2]
        output["Jet_pt"].fill(dataset=datalabel, pt=ak.fill_none(jet1.pt,-1), jetIndex='j1', cut='pt20cost0p8')
        output["Jet_pt"].fill(dataset=datalabel, pt=ak.fill_none(jet2.pt,-1), jetIndex='j2', cut='pt20cost0p8')
        output["Jet_pt"].fill(dataset=datalabel, pt=ak.fill_none(jet3.pt,-1), jetIndex='j3', cut='pt20cost0p8')
        output["Jet_eta"].fill(dataset=datalabel, eta=ak.fill_none(jet1.eta,1000), jetIndex='j1', cut='pt20cost0p8')
        output["Jet_eta"].fill(dataset=datalabel, eta=ak.fill_none(jet2.eta,1000), jetIndex='j2', cut='pt20cost0p8')
        output["Jet_eta"].fill(dataset=datalabel, eta=ak.fill_none(jet3.eta,1000), jetIndex='j3', cut='pt20cost0p8')
        output["Jet_phi"].fill(dataset=datalabel, phi=ak.fill_none(jet1.phi,1000), jetIndex='j1', cut='pt20cost0p8')
        output["Jet_phi"].fill(dataset=datalabel, phi=ak.fill_none(jet2.phi,1000), jetIndex='j2', cut='pt20cost0p8')
        output["Jet_phi"].fill(dataset=datalabel, phi=ak.fill_none(jet3.phi,1000), jetIndex='j3', cut='pt20cost0p8')
        output["Jet_mass"].fill(dataset=datalabel, mass=ak.fill_none(jet1.mass,-1), jetIndex='j1', cut='pt20cost0p8')
        output["Jet_mass"].fill(dataset=datalabel, mass=ak.fill_none(jet2.mass,-1), jetIndex='j2', cut='pt20cost0p8')
        output["Jet_mass"].fill(dataset=datalabel, mass=ak.fill_none(jet3.mass,-1), jetIndex='j3', cut='pt20cost0p8')
        output["Jet_energy"].fill(dataset=datalabel, energy=ak.fill_none(jet1.energy,-1), jetIndex='j1', cut='pt20cost0p8')
        output["Jet_energy"].fill(dataset=datalabel, energy=ak.fill_none(jet2.energy,-1), jetIndex='j2', cut='pt20cost0p8')
        output["Jet_energy"].fill(dataset=datalabel, energy=ak.fill_none(jet3.energy,-1), jetIndex='j3', cut='pt20cost0p8')
        
        # dR between (j1,j2)
        dr_j1j2 = jet1.delta_r(jet2)
        output["dR_j1j2"].fill(dataset=datalabel, dr=ak.fill_none(dr_j1j2,-1), cut='pt20cost0p8')
        
        
        
        
        
        return output

    def postprocess(self, accumulator):
        pass
    
p = MyProcessor()
output_gen = p.process(eve_gen, 'truth-level')
output_reco_sig = p.process(eve_reco_wobib, 'reco w/o BIB')
output_reco_sigbib = p.process(eve_reco_wbib, 'reco w/ BIB')